# SD-JWT for Certification of Registration credential in Proof of Business demo

## Setup cryptographic keys

In [1]:
import dataclasses
import asyncio
import json
from rich.console import Console
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

console = Console()
loop = asyncio.get_event_loop()

# Generate did:key identifier and associated public and privatekey pairs from the cryptographic seed.
cryptographic_seed = b'helloworld'
issuer_did = KeyDid(seed=cryptographic_seed)
issuer_did.create_keypair()

console.log('Issuer public key JWK:', issuer_did.public_key_jwk)

[20:24:45] Issuer public key JWK:                                                                  ]8;id=201548;file:///tmp/ipykernel_29926/3472563506.py\3472563506.py]8;;\:]8;id=208192;file:///tmp/ipykernel_29926/3472563506.py#15\15]8;;\
           {                                                                                                       
               'kty': 'EC',                                                                                        
               'kid': 'tJ7_O_Xu862O_9L4oxdBSkRotGJ2Id9smSovHtnUUo8',                                               
               'crv': 'P-256',                                                                                     
               'x': 'PTKECXSeSzIRRjYBKPECaRBRp1VH_g2EztKLJf8jpVc',                                                 
               'y': 'HHpPqdZ8P0ZVz9CSryp7FvmtTTnNImmlQuiatJncoNM'                                                  
           }                                                                                                       

## Claims to be issued

In [2]:
certificate_of_registration_claims = {
  "legalStatus": "ACTIVE",
  "legalForm": "Aktiebolag",
  "activity": "Real Estate",
  "orgNumber": "559133-2720",
  "registrationDate": "2005-10-08",
  "registeredAddress": {
    "fullAddress": "123 Main St",
    "thoroughFare": "Sveavägen",
    "postName": "Stockholm",
    "locatorDesignator": "111 34",
    "code": "48",
    "adminUnitLevel1": "SE"
  },
  "attestationValidity": "31-Dec-2022"
}

console.log("Certificate of Registration claims to be issued: ", certificate_of_registration_claims)

           Certificate of Registration claims to be issued:                                        ]8;id=462632;file:///tmp/ipykernel_29926/1992885474.py\1992885474.py]8;;\:]8;id=495120;file:///tmp/ipykernel_29926/1992885474.py#18\18]8;;\
           {                                                                                                       
               'legalStatus': 'ACTIVE',                                                                            
               'legalForm': 'Aktiebolag',                                                                          
               'activity': 'Real Estate',                                                                          
               'orgNumber': '559133-2720',                                                                         
               'registrationDate': '2005-10-08',                                                                   
               'registeredAddress': {                                                                              
                   'fullAddress': '123 Main St',                                                                   
                   'thoroughFare': 'Sveavägen',                                                                    
                   'postName': 'Stockholm',                                                                        
                   'locatorDesignator': '111 34',                                                                  
                   'code': '48',                                                                                   
                   'adminUnitLevel1': 'SE'                                                                         
               },                                                                                                  
               'attestationValidity': '31-Dec-2022'                                                                
           }                                                                                                       

## Calculate full disclosure for claims to be issued

In [3]:
import json
import base64

disclosure = base64.urlsafe_b64encode(json.dumps(certificate_of_registration_claims).encode('utf-8')).decode('utf-8').rstrip("=")
console.log("Disclosure: ", disclosure)

           Disclosure:                                                                              ]8;id=360829;file:///tmp/ipykernel_29926/2974318599.py\2974318599.py]8;;\:]8;id=631420;file:///tmp/ipykernel_29926/2974318599.py#5\5]8;;\
           eyJsZWdhbFN0YXR1cyI6ICJBQ1RJVkUiLCAibGVnYWxGb3JtIjogIkFrdGllYm9sYWciLCAiYWN0aXZpdHkiOiAi                
           UmVhbCBFc3RhdGUiLCAib3JnTnVtYmVyIjogIjU1OTEzMy0yNzIwIiwgInJlZ2lzdHJhdGlvbkRhdGUiOiAiMjAw                
           NS0xMC0wOCIsICJyZWdpc3RlcmVkQWRkcmVzcyI6IHsiZnVsbEFkZHJlc3MiOiAiMTIzIE1haW4gU3QiLCAidGhv                
           cm91Z2hGYXJlIjogIlN2ZWF2XHUwMGU0Z2VuIiwgInBvc3ROYW1lIjogIlN0b2NraG9sbSIsICJsb2NhdG9yRGVz                
           aWduYXRvciI6ICIxMTEgMzQiLCAiY29kZSI6ICI0OCIsICJhZG1pblVuaXRMZXZlbDEiOiAiU0UifSwgImF0dGVz                
           dGF0aW9uVmFsaWRpdHkiOiAiMzEtRGVjLTIwMjIifQ                                                              

## Calculate disclosure SHA-256 digest

In [4]:
import hashlib
import base64

sha256_hash = hashlib.sha256()
sha256_hash.update(disclosure.encode('utf-8'))
sha256_digest = sha256_hash.digest()

disclosure_sha_256 = base64.urlsafe_b64encode(sha256_digest).decode('utf-8').rstrip("=")
console.log("Disclosure SHA-256: ", disclosure_sha_256)

           Disclosure SHA-256:  JhDIF1oJsAhyPf4p9p97h6iSEH5Nfbj7eq_L8tXHfsY                         ]8;id=669524;file:///tmp/ipykernel_29926/2709858298.py\2709858298.py]8;;\:]8;id=188720;file:///tmp/ipykernel_29926/2709858298.py#9\9]8;;\

## Create SD-JWT

In [10]:
import json
from jwcrypto import jwt

_sd = [disclosure_sha_256]
sd_jwt = issuer_did.generate_sd_jwt(_sd)

console.log("SD-JWT: ", sd_jwt)

sd_jwt_deserialised = jwt.JWT(key=issuer_did._key, jwt=sd_jwt)
console.log("SD-JWT payload: ", json.loads(sd_jwt_deserialised.claims))
console.log("SD-JWT header: ",  json.loads(sd_jwt_deserialised.header))

[20:30:20] SD-JWT:                                                                                  ]8;id=404611;file:///tmp/ipykernel_29926/1335579605.py\1335579605.py]8;;\:]8;id=691273;file:///tmp/ipykernel_29926/1335579605.py#7\7]8;;\
           eyJhbGciOiJFUzI1NiJ9.eyJfc2QiOlsiSmhESUYxb0pzQWh5UGY0cDlwOTdoNmlTRUg1TmZiajdlcV9MOHRYSGZ                
           zWSJdLCJfc2RfYWxnIjoic2hhLTI1NiIsImNuZiI6eyJqd2siOnsiY3J2IjoiUC0yNTYiLCJraWQiOiJ0SjdfT19                
           YdTg2Mk9fOUw0b3hkQlNrUm90R0oySWQ5c21Tb3ZIdG5VVW84Iiwia3R5IjoiRUMiLCJ4IjoiUFRLRUNYU2VTekl                
           SUmpZQktQRUNhUkJScDFWSF9nMkV6dEtMSmY4anBWYyIsInkiOiJISHBQcWRaOFAwWlZ6OUNTcnlwN0Z2bXRUVG5                
           OSW1tbFF1aWF0Sm5jb05NIn19LCJleHAiOjE2ODcyNzY4MjAsImlhdCI6MTY4NzI3MzIyMCwiaXNzIjoiaHR0cHM                
           6Ly9pc3N1ZXIuaWdyYW50LmlvIn0.Qny6zBvDu5GKgDz52-06oEl4y2UUDhzLhoPSt7DDuD76Yxdwe7crOfoCw4D                
           A73qgy8uNFAnN98hsZ4m8rzgXJw                                                                             

           SD-JWT payload:                                                                         ]8;id=572119;file:///tmp/ipykernel_29926/1335579605.py\1335579605.py]8;;\:]8;id=780867;file:///tmp/ipykernel_29926/1335579605.py#10\10]8;;\
           {                                                                                                       
               '_sd': ['JhDIF1oJsAhyPf4p9p97h6iSEH5Nfbj7eq_L8tXHfsY'],                                             
               '_sd_alg': 'sha-256',                                                                               
               'cnf': {                                                                                            
                   'jwk': {                                                                                        
                       'crv': 'P-256',                                                                             
                       'kid': 'tJ7_O_Xu862O_9L4oxdBSkRotGJ2Id9smSovHtnUUo8',                                       
                       'kty': 'EC',                                                                                
                       'x': 'PTKECXSeSzIRRjYBKPECaRBRp1VH_g2EztKLJf8jpVc',                                         
                       'y': 'HHpPqdZ8P0ZVz9CSryp7FvmtTTnNImmlQuiatJncoNM'                                          
                   }                                                                                               
               },                                                                                                  
               'exp': 1687276820,                                                                                  
               'iat': 1687273220,                                                                                  
               'iss': 'https://issuer.igrant.io'                                                                   
           }                                                                                                       

           SD-JWT header:                                                                          ]8;id=525943;file:///tmp/ipykernel_29926/1335579605.py\1335579605.py]8;;\:]8;id=895992;file:///tmp/ipykernel_29926/1335579605.py#11\11]8;;\
           {'alg': 'ES256'}                                                                                        

## Create combined format for issuance (a.k.a issue credential to holder)

*Note: Format is `<SD-JWT>~<Disclosure 1>~<Disclosure 2>~...~<Disclosure N>`*

In [11]:
combined_format_for_issuance = sd_jwt + "~" + disclosure_sha_256
console.log("Combined format for issuance : ", combined_format_for_issuance)

[20:31:52] Combined format for issuance (<SD-JWT>~<Disclosure 1>~<Disclosure 2>~...~<Disclosure N>) ]8;id=977527;file:///tmp/ipykernel_29926/3174722539.py\3174722539.py]8;;\:]8;id=734158;file:///tmp/ipykernel_29926/3174722539.py#2\2]8;;\
           :                                                                                                       
           eyJhbGciOiJFUzI1NiJ9.eyJfc2QiOlsiSmhESUYxb0pzQWh5UGY0cDlwOTdoNmlTRUg1TmZiajdlcV9MOHRYSGZ                
           zWSJdLCJfc2RfYWxnIjoic2hhLTI1NiIsImNuZiI6eyJqd2siOnsiY3J2IjoiUC0yNTYiLCJraWQiOiJ0SjdfT19                
           YdTg2Mk9fOUw0b3hkQlNrUm90R0oySWQ5c21Tb3ZIdG5VVW84Iiwia3R5IjoiRUMiLCJ4IjoiUFRLRUNYU2VTekl                
           SUmpZQktQRUNhUkJScDFWSF9nMkV6dEtMSmY4anBWYyIsInkiOiJISHBQcWRaOFAwWlZ6OUNTcnlwN0Z2bXRUVG5                
           OSW1tbFF1aWF0Sm5jb05NIn19LCJleHAiOjE2ODcyNzY4MjAsImlhdCI6MTY4NzI3MzIyMCwiaXNzIjoiaHR0cHM                
           6Ly9pc3N1ZXIuaWdyYW50LmlvIn0.Qny6zBvDu5GKgDz52-06oEl4y2UUDhzLhoPSt7DDuD76Yxdwe7crOfoCw4D                
           A73qgy8uNFAnN98hsZ4m8rzgXJw~JhDIF1oJsAhyPf4p9p97h6iSEH5Nfbj7eq_L8tXHfsY                                 